## Midas touch されそうな銘柄を探す
+ [ミダース - Wikipedia](https://ja.wikipedia.org/wiki/%E3%83%9F%E3%83%80%E3%83%BC%E3%82%B9)
    + ギリシア神話の中でミダース（古希: Μίδας, Midās）は、プリュギア（Phrygia）の都市ペシヌス（Pessinus）の王。長母音を省略してミダスとも表記される。触ったもの全てを黄金に変える能力（"Midas touch"）のため広く知られている。
    
+ 埋蔵量が多い
+ 今まではコストが高かったので株価が低く評価されていた銘柄
+ カントリーリスクがなるべく無い


### Gold Miners に関するメモ

+ [RANKED: Top 10 lowest cost gold mines in the world Q1 - MINING.COM](https://www.mining.com/featured-article/ranked-top-10-lowest-cost-gold-mines-in-the-world-q1/)
    + AISC指標は、鉱山の操業効率のベンチマーク
    + 鉱山の経済性


In [ ]:
import numpy as np

# Import Pipeline class and datasets
from quantopian.pipeline import Pipeline
from quantopian.pipeline.data import USEquityPricing

from quantopian.pipeline.data.morningstar import Fundamentals

from quantopian.pipeline.data.factset import Fundamentals as factst_funda
from quantopian.pipeline.filters import Q500US, StaticSids, StaticAssets

# Import built-in trading universe
from quantopian.pipeline.filters import QTradableStocksUS

# The 10 Biggest Gold Mining Stocks
# https://www.fool.com/investing/the-10-biggest-gold-mining-stocks.aspx
    
gold_tickers = ['AEM', 'AGI', 'ALO', 'AU', 'BVN', 'DRD', 'EGO',
                'FCX',  'GFI', 'GOLD', 'HMY', 'KGC', 'KL',
                 'NEM',  'SBSW'] #'NG','GDX', 'GLD'

def make_pipeline():
    # Create a reference to our trading universe
    base_universe = QTradableStocksUS()

    # Get latest closing price
    close_price = USEquityPricing.close.latest
    
    #current_liabilities
    #The debts or obligations of the firm that are due within one year.
    current_liabilities = Fundamentals.current_liabilities.latest
    
    #total_assets
    #The aggregate amount of probable future economic benefits obtained or controlled by a particular enterprise as a result of past transactions or events.
    total_assets = Fundamentals.total_assets.latest
    market_cap = Fundamentals.market_cap.latest
    morningstar_industry_group_code = Fundamentals.morningstar_industry_group_code.latest
    morningstar_industry_code = Fundamentals.morningstar_industry_code.latest
    total_liabilities_net_minority_interest = Fundamentals.total_liabilities_net_minority_interest.latest
    operating_income = Fundamentals.operating_income.latest
    interest_expense = Fundamentals.interest_expense.latest
    cash_and_cash_equivalents = Fundamentals.cash_and_cash_equivalents.latest
    current_assets = Fundamentals.current_assets.latest
    current_liabilities = Fundamentals.current_liabilities.latest
    current_ratio = Fundamentals.current_ratio.latest
    
    enterprise_value = Fundamentals.enterprise_value.latest
    ebitda = Fundamentals.ebitda.latest
    enterprise_multiple = enterprise_value / ebitda

    
    # interest_expense

    myassets = StaticAssets(symbols(gold_tickers + ["GLD"]))
    
    # Return Pipeline containing close_price and
    # sentiment_score that has our trading universe as screen
    return Pipeline(
        columns={
            'close_price': close_price,
            'enterprise_multiple':enterprise_multiple,
#             'current_liabilities':current_liabilities, 
#             'total_assets':total_assets, 
#             'cash_and_cash_equivalents':cash_and_cash_equivalents,
#             'current_assets':current_assets,
#             'liab/asset':total_liabilities_net_minority_interest/total_assets,
#             'cash/asset':cash_and_cash_equivalents/total_assets,
#             'market_cap':market_cap / 1000000000,
#             'morningstar_industry_group_code':morningstar_industry_group_code,
#             'morningstar_industry_code':morningstar_industry_code,
#             'total_liabilities_net_minority_interest': total_liabilities_net_minority_interest, 
#             'operating_income':operating_income, 
#             'interest_expense':interest_expense,
#             'current_liabilities':current_liabilities, 
#             'current_ratio':current_ratio,
            
        },
        screen=myassets #base_universe #myassets
    )

# Import run_pipeline method
from quantopian.research import run_pipeline

# Execute pipeline created by make_pipeline
# between start_date and end_date
pipeline_output = run_pipeline(
    make_pipeline(),
    start_date='2015-09-01',
    end_date='2020-05-01'
)

# Display last 10 rows
pipeline_output.tail(10)

In [ ]:
pipeline_output["enterprise_multiple"].unstack().describe().T.sort_values(by="mean")

In [ ]:
pipeline_output["enterprise_multiple"].unstack().plot()

In [ ]:
x = gold_tickers.copy()
n = len(x)
yoko = 3
tate = int(n / yoko) + 1 
print(tate, yoko)

#pipeline_output["enterprise_multiple"].unstack().plot(subplots=True, layout=(tate, yoko), figsize=(20,20), ylim=(-100,200))

In [ ]:
import matplotlib.pyplot as plt 

fig = plt.figure(figsize=(20,30))
plt.subplots_adjust()

for i, ticker in enumerate(gold_tickers):
    ax = fig.add_subplot(tate, yoko, i+1)
    ax.set_title(ticker)
    ax1 = ax.twinx()
    ax.plot(pipeline_output["enterprise_multiple"].xs(symbols(ticker), level=1),color="#002a83")
    ax.set_ylim(-100,200)
    ax1.plot(pipeline_output["close_price"].xs(symbols("GLD"), level=1), color="#e50000", alpha=0.3)
    
